In [1]:
import pandas as pd
import glob
import os

In [2]:
from IPython.display import HTML, display
from cellphonedb.utils import db_releases_utils

display(HTML(db_releases_utils.get_remote_database_versions_html()['db_releases_html_table']))

Version,Release date,v4.1.0,2023-03-09


In [3]:
# -- Version of the databse
cpdb_version = 'v4.1.0'

# -- Path where the input files to generate the database are located
cpdb_target_dir = os.path.join('/QRISdata/Q6104/Charlotte', cpdb_version)

In [4]:
from cellphonedb.utils import db_utils

db_utils.download_database(cpdb_target_dir, cpdb_version)

In [ ]:
import os
os.getcwd()

'/home/s4735100'

In [ ]:
# we just downloaded this file from above
cpdb_file_path = "/QRISdata/Q6104/Charlotte/v4.1.0/cellphonedb.zip"
# we will create this one from our adata.obs
meta_file_path = "/scratch/user/s4735100/metadata.tsv"
# this is our adata .h5ad file
count_file_path = "/scratch/user/s4735100/countcpdb.h5ad"
# microenvs_file_path just ignore
out_path = "/scratch/user/s4735100/cellphonedb_out"

In [ ]:
## so first we will read in the single cell object
import scanpy as sc

adata = sc.read_h5ad("/scratch/user/s4735100/harmony.h5ad")
adata

AnnData object with n_obs × n_vars = 89927 × 1393
    obs: 'patient_id', 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'leiden', 'leiden_colors', 'leiden_sizes', 'log1p', 'neighbors', 'paga', 'patient_id_colors', 'pca', 'rank_genes_groups', 'sample_id_colors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [ ]:
adata2 = adata.raw.to_adata()
adata2

AnnData object with n_obs × n_vars = 89927 × 24842
    obs: 'patient_id', 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'leiden', 'leiden_colors', 'leiden_sizes', 'log1p', 'neighbors', 'paga', 'patient_id_colors', 'pca', 'rank_genes_groups', 'sample_id_colors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    obsp: 'connectivities', 'distances'

In [ ]:
adata2.write_h5ad(count_file_path, compression="gzip")

In [ ]:
metadata = pd.DataFrame(adata2.obs["leiden"]).reset_index()
metadata.columns = ["barcode_sample", "cell_type"]
metadata

,barcode_sample,cell_type
0,AAACCTGAGCGTCAAG-1,RPC1
1,AAACCTGAGTGACTCT-1,RPC1
2,AAACCTGCAAGAAGAG-1,unknown2
3,AAACCTGCACTGTCGG-1,RPC3
4,AAACCTGCAGATAATG-1,Interneuron
...,...,...
89922,TTTGTCATCGATGAGG-1-1,RPC4
89923,TTTGTCATCTCAAACG-1,RPC6
89924,TTTGTCATCTCATTCA-1-1,CP1
89925,TTTGTCATCTCGCATC-1,RPC7


In [ ]:
metadata.to_csv(meta_file_path, sep="\t", index=False)

In [ ]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method

deconvoluted, means, pvalues, significant_means = cpdb_statistical_analysis_method.call(
    cpdb_file_path = cpdb_file_path,                 # mandatory: CellPhoneDB database zip file.
    meta_file_path = meta_file_path,                 # mandatory: tsv file defining barcodes to cell label.
    counts_file_path = count_file_path,             # mandatory: normalized count matrix.
    counts_data = 'hgnc_symbol',                     # defines the gene annotation in counts matrix.
    iterations = 1000,                               # denotes the number of shufflings performed in the analysis.
    threshold = 0.1,                                 # defines the min % of cells expressing a gene for this to be employed in the analysis.
    threads = 2,                                     # number of threads to use in the analysis.
    debug_seed = 42,                                 # debug randome seed. To disable >=0.
    result_precision = 3,                            # Sets the rounding for the mean values in significan_means.
    pvalue = 0.05,                                   # P-value threshold to employ for significance.
    subsampling = False,                             # To enable subsampling the data (geometri sketching).
    subsampling_log = False,                         # (mandatory) enable subsampling log1p for non log-transformed data inputs.
    subsampling_num_pc = 100,                        # Number of componets to subsample via geometric skectching (dafault: 100).
    subsampling_num_cells = 1000,                    # Number of cells to subsample (integer) (default: 1/3 of the dataset).
    separator = '|',                                 # Sets the string to employ to separate cells in the results dataframes "cellA|CellB".
    debug = False,                                   # Saves all intermediate tables employed during the analysis in pkl format.
    output_path = out_path,                          # Path to save results.
    output_suffix = None                             # Replaces the timestamp in the output files by a user defined string in the  (default: None).
    )


Reading user files...
The following user files were loaded successfully:
/scratch/user/s4735100/countcpdb.h5ad
/scratch/user/s4735100/metadata.tsv
[ ][CORE][22/09/23-14:32:10][INFO] [Cluster Statistical Analysis] Threshold:0.1 Iterations:1000 Debug-seed:42 Threads:2 Precision:3
[ ][CORE][22/09/23-14:32:10][WARNING] Debug random seed enabled. Set to 42
[ ][CORE][22/09/23-14:32:15][INFO] Running Real Analysis
[ ][CORE][22/09/23-14:32:15][INFO] Running Statistical Analysis


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [15:16<00:00,  1.09it/s]

[ ][CORE][22/09/23-14:47:32][INFO] Building Pvalues result


[ ][CORE][22/09/23-14:47:32][INFO] Building results
Saved deconvoluted to /scratch/user/s4735100/cellphonedb_out/statistical_analysis_deconvoluted_09_22_2023_14:47:32.txt
Saved means to /scratch/user/s4735100/cellphonedb_out/statistical_analysis_means_09_22_2023_14:47:32.txt
Saved pvalues to /scratch/user/s4735100/cellphonedb_out/statistical_analysis_pvalues_09_22_2023_14:47:32.txt
Saved significant_means to /scratch/user/s4735100/cellphonedb_out/statistical_analysis_significant_means_09_22_2023_14:47:32.txt


In [ ]:
import ktplotspy as kpy
import matplotlib.pyplot as plt